In [1]:
_ = ! pip install web3

In [2]:
import os, sys

from datetime import datetime, timedelta
import time

import requests, json
from web3 import Web3, HTTPProvider

from eth_utils import to_bytes
from eth_keys import keys
from eth_account import Account
from eth_account.messages import SignableMessage

In [3]:
w3 = Web3(HTTPProvider("https://rpc.ankr.com/eth/1e5f9fd3e49ecec581e00815ea1eb41bc459248fd3dcf67716ff3237cc3ab68b"))
# https://1rpc.io/eth

In [4]:
source_token = "0x1f9840a85d5af5bf1d1762f925bdaddc4201f984" # UniSwap
destination_token = "0xdac17f958d2ee523a2206206994597c13d831ec7" # USDT

source_amount = 1000000000000000000
slippage = int(0.05 * 10000) # Convert to bps

In [28]:
eth_wallet = Account.from_key("e5ecbcfc71ad24a2131953ef9a66e4b7ed8ca9a10ffd92104e106fab1f0af4f7")

In [31]:
# 0.  One fresh quote – DO NOT fetch another one before sending
quote  = requests.get(
    "https://api.0x.org/swap/allowance-holder/quote",
    params={
        "chainId": 1,
        "sellToken": source_token,
        "buyToken":  destination_token,
        "sellAmount": source_amount,
        "taker": eth_wallet.address,
        "slippageBps": slippage,
    },
    headers={
        "0x-version" : "v2",
        "0x-api-key" : "50cbb531-4fda-4464-acb4-5e6b5bf0e2d6"
    },
    timeout=10,
).json()

In [30]:
ERC20_ABI: list[dict] = [
    {
        "constant": True,
        "inputs": [{"name": "owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "type": "function",
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [{"name": "", "type": "uint8"}],
        "type": "function",
    },
]

erc20 = w3.eth.contract(address=Web3.to_checksum_address(source_token), abi=ERC20_ABI)

sell_amount   = int(quote["sellAmount"])
permit_amount = int(quote["permit2"]["eip712"]["message"]["permitted"]["amount"])
owner_balance = int(erc20.functions.balanceOf(eth_wallet.address).call())

print ("sellAmount        :", sell_amount)
print ("permit.amount     :", permit_amount)
print ("wallet UNI balance:", owner_balance)
print ()
print ("gas fee:", (1.0e-18) * int(quote['transaction']['gas']) * int(quote['transaction']['gasPrice']), "ETH")

KeyError: 'permit2'

In [19]:
HEAD_IDX = 9  # from your dump: head[9] -> ptr=544

def build_permit2_calldata(quote, wallet_key: bytes) -> str:
    data = bytes.fromhex(quote["transaction"]["data"][2:])
    head_pos = 4 + HEAD_IDX * 32

    # 1) sign raw 32-byte digest
    digest  = bytes.fromhex(quote["permit2"]["hash"][2:])
    sig_obj = keys.PrivateKey(wallet_key).sign_msg_hash(digest)
    sig     = sig_obj.to_bytes()
    if sig[-1] in (0, 1):
        sig = sig[:-1] + bytes([sig[-1] + 27])
    assert len(sig) == 65

    # 2) find the ONLY 97-zero block we can safely overwrite
    ZERO97 = b"\x00" * 97
    off97  = data.rfind(ZERO97)
    if off97 == -1:
        raise RuntimeError("No 97-byte zero placeholder anywhere")

    # 3) write (65 || sig) there
    patched = data[:off97] + (65).to_bytes(32, "big") + sig + data[off97 + 97:]

    # 4) repoint head[9] to our new block
    new_ptr = off97 - 4  # pointer is relative to start of body
    patched = (
        patched[:head_pos]
        + new_ptr.to_bytes(32, "big")
        + patched[head_pos + 32:]
    )

    # 5) prove what the router will see
    ptr     = int.from_bytes(patched[head_pos:head_pos+32], "big")
    abs_off = 4 + ptr
    L = int.from_bytes(patched[abs_off:abs_off+32], "big")
    print("ptr:", ptr, "abs_off:", abs_off, "len_at_ptr:", L)
    assert L == 65, "router will still NOT see 65"

    return "0x" + patched.hex()

# ---- use it ----
calldata = build_permit2_calldata(quote, eth_wallet.key)

ptr: 1245 abs_off: 1249 len_at_ptr: 65


In [32]:
# 4. Build, sign, send
tx: dict = {
    "chainId":  1,
    "to":       Web3.to_checksum_address(quote["transaction"]["to"]),
    "data":     quote["transaction"]["data"], # calldata,
    "value":    int(quote["transaction"]["value"]),
    "gas":      int(quote["transaction"]["gas"]),
    "nonce":    w3.eth.get_transaction_count(eth_wallet.address, "pending"),
}

# Prefer EIP-1559 if present
if ("maxFeePerGas" in quote["transaction"]) and ("maxPriorityFeePerGas" in quote["transaction"]):
    tx["type"] = 2
    tx["maxFeePerGas"] = int(quote["transaction"]["maxFeePerGas"])
    tx["maxPriorityFeePerGas"] = int(quote["transaction"]["maxPriorityFeePerGas"])
else:
    tx["gasPrice"] = int(quote["transaction"]["gasPrice"])

# Pass through optional fields
if "accessList" in quote["transaction"]:
    tx["accessList"] = quote["transaction"]["accessList"]

# Only for eth_call / estimateGas preflight
tx = dict(tx, **{"from": eth_wallet.address})

tx_signed = eth_wallet.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(tx_signed.raw_transaction)

In [33]:
# 5. Check result
tx_hex = w3.to_hex(tx_hash)

print("↗ broadcast :", tx_hex)
print("⏳ waiting …")
print("✅ status   :", w3.eth.wait_for_transaction_receipt(tx_hash).status)  # 1 = success

↗ broadcast : 0x02c63211902d4c300730ec4d507319914df6ba975bf2b26986a4eeeff21099da
⏳ waiting …
✅ status   : 0


In [34]:
print ("https://eth.blockscout.com/tx/" + tx_hex)

https://eth.blockscout.com/tx/0x02c63211902d4c300730ec4d507319914df6ba975bf2b26986a4eeeff21099da


In [35]:
from web3 import Web3

def diag_allowance(q, token, wallet, w3):
    txo = q.get("transaction", q)
    allowance_target = q.get("allowanceTarget") \
        or (q.get("issues", {}).get("allowance", {}) or {}).get("spender")
    router_to = txo["to"]

    allowance_target = Web3.to_checksum_address(allowance_target) if allowance_target else None
    router_to        = Web3.to_checksum_address(router_to)

    print("router to:        ", router_to)
    print("allowanceTarget:  ", allowance_target, "<-- MUST approve this address")
    assert allowance_target, "No allowanceTarget in quote – this is required for allowance-holder flow."

    erc20 = w3.eth.contract(address=Web3.to_checksum_address(token), abi=[
        {"constant":True,"inputs":[{"name":"owner","type":"address"},{"name":"spender","type":"address"}],
         "name":"allowance","outputs":[{"name":"","type":"uint256"}],"type":"function"},
        {"constant":True,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"type":"function"},
    ])
    dec = erc20.functions.decimals().call()
    cur = erc20.functions.allowance(wallet, allowance_target).call()

    print(f"decimals: {dec}")
    print(f"current allowance (raw): {cur}")
    return allowance_target, cur

# Example call (after you got `quote`):
# spender, cur_allow = diag_allowance(quote, SELL_TOKEN, WALLET, w3)

In [36]:
diag_allowance(quote, source_token, eth_wallet.address, w3)

router to:         0x0000000000001fF3684f28c67538d4D072C22734
allowanceTarget:   0x0000000000001fF3684f28c67538d4D072C22734 <-- MUST approve this address
decimals: 18
current allowance (raw): 0


('0x0000000000001fF3684f28c67538d4D072C22734', 0)

In [38]:
PERMIT2 = Web3.to_checksum_address("0x000000000022d473030f116ddee9f6b43ac78ba3")
AH      = Web3.to_checksum_address("0x0000000000001fF3684f28c67538d4D072C22734")  # from quote
owner   = Web3.to_checksum_address(eth_wallet.address)
token   = Web3.to_checksum_address(source_token)

erc20 = w3.eth.contract(address=token, abi=[
  {"constant":True,"inputs":[{"name":"owner","type":"address"},{"name":"spender","type":"address"}],
   "name":"allowance","outputs":[{"name":"","type":"uint256"}],"type":"function"},
])

print("allow(PERMIT2):", erc20.functions.allowance(owner, PERMIT2).call())
print("allow(AHolder):", erc20.functions.allowance(owner, AH).call())

allow(PERMIT2): 0
allow(AHolder): 0
